## Optimize project cost given project constraints. Formulate it as constraint optimization problem and solve it.

## Problem Description

#### In project management, we frequently encounter requirements to plan project such that overall project is completed within committed time to customers/users, but with minimal cost. This is clearly a optimization problem. Below we formulate it as Linear Programming(LP) problem and resolve it using PuLP, the python LP solver.

### <span style="color:red">Aim</span>
### <span style="color:green">We want optimize/minimize the project cost such that project duration is within K weeks</span>

#### Project tasks, their inter dependencies, normal durations, normal costs, crashing durations and crashing costs are given. 

#### Crashing means max extent to which task duration can be decreased, but with increased costs. For eg: A task with normal duration of 1 week with normal cost of 100 dollars, might be decreased to 0.5 weeks(crashing duration) with a (crashing) cost increased to 160 dollars.

#### Tools used: 
1. PuLP (python Linear programming library) with CBC and GLPK backend solvers.
2. Numpy
3. Pandas

In [1]:
# pip install pulp 
# sudo apt-get install coinor-cbc glpk-utils

In [2]:
import pandas as pd
import numpy as np

##### Import project tasks, their inter dependencies, normal durations, normal costs, crashing durations and crashing costs. This data is taken from https://www.linearprogramming.info/formulation-and-resolution-of-a-model-of-linear-programming-to-reduce-the-duration-of-a-project-crashing/

In [3]:
# task_details is stored in pandas dataframe df
df = pd.read_csv("./task_details.csv",index_col = ['task'])
df

,prerequisite_tasks,normal_duration_weeks,normal_cost_dollars,crashing_duration_weeks,crashing_cost_dollars
task,,,,,
A,,1.0,100,0.5,140
B,A,3.0,150,1.0,270
C,A,1.0,120,0.5,160
D,"B,C",1.0,10,1.0,10
E,D,1.0,225,0.5,300
F,D,3.0,500,2.0,700
G,D,3.0,400,1.0,500
H,G,1.0,150,0.5,170
I,"F,H",1.5,75,0.5,135


In above, critical path is A-B-D-G-H-I-K-L.

In [4]:
normal_project_cost = np.sum(df['normal_cost_dollars'])
print "Normal Project cost is ", normal_project_cost, " dollars"

Normal Project cost is  2620  dollars


In [5]:
# tasks in critical path
critical_tasks = ['A','B','D','G','H','I','K','L']
df1 = df.loc[df.index.isin(critical_tasks)]
normal_project_duration = np.sum(df1['normal_duration_weeks'])
print "Normal Project duration is ", normal_project_duration, " weeks"

Normal Project duration is  15.5  weeks


## Pre-process and augment input data with useful parameters

#### 1. Add a fictituous task M to mark completion of project. i.e task M has dependency on all other tasks and with 0 cost and 0 duration

In [6]:
df.loc['M'] = ["A,B,C,D,E,F,G,H,I,J,K,L", 0, 0, 0, 0]
df.loc['M']

prerequisite_tasks         A,B,C,D,E,F,G,H,I,J,K,L
normal_duration_weeks                            0
normal_cost_dollars                              0
crashing_duration_weeks                          0
crashing_cost_dollars                            0
Name: M, dtype: object

#### 2. Compute crash cost per week for each task i.e extra cost if duration is reduced by one week.

In [7]:
df['crash_cost_per_week'] =(df['crashing_cost_dollars'] - df['normal_cost_dollars']) / \
                           (df['normal_duration_weeks'] - df['crashing_duration_weeks'])

df['crash_cost_per_week'] = df['crash_cost_per_week'].fillna(0)
# Assert crash cost per week for task B and M. It should be 60 and 0 respectively.
df.loc['B']['crash_cost_per_week'] == 60 and df.loc['M']['crash_cost_per_week'] == 0

True

#### 3. Compute max duration by which task duration can be reduced. 

For eg: task G can be reduced by 2 weeks to crashing duration of 1 week.

In [8]:
df['max_reduction_duration_weeks'] = df['normal_duration_weeks'] - df['crashing_duration_weeks']
# Assert max reduction possible for task G - it should be 2.0 weeks.
df.loc['G']['max_reduction_duration_weeks'] == 2

True

## Identify the decision variables

Since we have a constraint that project duration should be within K weeks, we need to decide which task starts when and which we need to crash. 

s_i = week at which task i is started. For eg: task B is started at week 1

r_i = weeks by which task i is reduced with max reduction being crashing_duration_weeks.<br>&nbsp;&nbsp;&nbsp;&nbsp;
      For eg: Duration of task B can be reduced from 3.0 to 1.0 (i.e r_i value = 2.0) with crashing cost of 270$.

i in ('A' to 'M')

### Other variables

n_i = normal duration of task i

mr_i = max reduction in weeks possible for task i

ccpw_i = crashing cost per week of task i

K = max duration in weeks, constraint given for project completion


## Formulate the objective function

Minimize the extra cost of reducing tasks duration i.e 

sum (ccpw_i * r_i)   for all tasks i in A to M

## The constraints

1. s_M <= K  i.e task M should be started and completed(since duration is 0) within K weeks to mark project completion.

2. r_i <= rm_i  i.e task i can be reduced at max by rm_i weeks.

3. s_B >= n_A + s_A - r_A , task B can be started only after task A is completed

4. s_C >= n_A + s_A - r_A , task C can be started only after task A is completed

5. s_D >= n_B + s_B - r_B , task D can be started only after task B is completed

6. s_D >= n_C + s_C - r_C , task D can be started only after task C is completed

7. s_E >= n_D + s_D - r_D , task E can be started only after task D is completed

8. s_F >= n_D + s_D - r_D , task F can be started only after task D is completed

9. s_G >= n_D + s_D - r_D , task G can be started only after task D is completed

10. s_H >= n_G + s_G - r_G , task H can be started only after task G is completed

11. s_I >= n_F + s_F - r_F , task I can be started only after task F is completed

12. s_I >= n_H + s_H - r_H , task I can be started only after task H is completed

13. s_J >= n_I + s_I - r_I , task J can be started only after task I is completed

14. s_K >= n_I + s_I - r_I , task K can be started only after task I is completed

15. s_L >= n_K + s_K - r_K , task L can be started only after task K is completed

16. s_M >= n_i + s_i - r_i , task M can be started only after all tasks i are completed


Decision variables are non-negative.

1. s_i >= 0  i.e week at which task i starts should be non-negative week

2. r_i >= 0  i.e task i can be reduced by non-negative weeks

## Solving using PuLP LP python solver

In [9]:
import pulp
prob = pulp.LpProblem(name = "Reducing project cost", sense = pulp.LpMinimize)

#max duration in weeks, constraint/deadline given for project completion
K = 9.5

In [10]:
# Decision variables
s = pulp.LpVariable.dicts(name ="s", indexs = (df.index) , lowBound=0, cat = 'Continuous')
s

{'A': s_A,
 'B': s_B,
 'C': s_C,
 'D': s_D,
 'E': s_E,
 'F': s_F,
 'G': s_G,
 'H': s_H,
 'I': s_I,
 'J': s_J,
 'K': s_K,
 'L': s_L,
 'M': s_M}

In [11]:
r = pulp.LpVariable.dicts(name = 'r', indexs = (df.index), lowBound=0, cat='Integer')
r

{'A': r_A,
 'B': r_B,
 'C': r_C,
 'D': r_D,
 'E': r_E,
 'F': r_F,
 'G': r_G,
 'H': r_H,
 'I': r_I,
 'J': r_J,
 'K': r_K,
 'L': r_L,
 'M': r_M}

In [12]:
# Objective function
# sum of ccpw_i * r_i for all tasks i in A to M
prob += pulp.lpSum([r[task] * df.loc[task]['crash_cost_per_week'] \
                    for task in df.index])

In [13]:
# Constraints
prob += s['M'] <= K, "task M should be started and completed within K="+ str(K) + " weeks."

for task in df.index:
    rm_i = df.loc[task]['max_reduction_duration_weeks']
    n_i  = df.loc[task]['normal_duration_weeks']
    dep  = df.loc[task]['prerequisite_tasks']
    
    prob += r[task] <= rm_i, "task " + task + " can be reduced at max by " + str(rm_i) + " weeks."
    prob += s[task] >= 0   , "week at which task " + task + " starts should be non-negative "
    prob += r[task] >= 0   , "task " + task + " can be reduced by non-negative weeks"
        
    # Clean up prequisite tasks and parse it.    
    if dep.strip() != '' and dep.strip() != ',':
        dep_tasks = dep.split(',')
        for dep_i in dep_tasks:
            prob +=  s[task] >= df.loc[dep_i]['normal_duration_weeks'] + s[dep_i] -r[dep_i],\
                    "Task " + task + " can be started only after task " + dep_i + " is completed"


In [14]:
print(prob)

Reducing project cost:
MINIMIZE
80.0*r_A + 60.0*r_B + 80.0*r_C + 150.0*r_E + 200.0*r_F + 50.0*r_G + 40.0*r_H + 60.0*r_I + 70.0*r_J + 90.0*r_K + 140.0*r_L + 0.0
SUBJECT TO
task_M_should_be_started_and_completed_within_K=9.5_weeks.: s_M <= 9.5

task_A_can_be_reduced_at_max_by_0.5_weeks.: r_A <= 0.5

week_at_which_task_A_starts_should_be_non_negative_: s_A >= 0

task_A_can_be_reduced_by_non_negative_weeks: r_A >= 0

task_B_can_be_reduced_at_max_by_2.0_weeks.: r_B <= 2

week_at_which_task_B_starts_should_be_non_negative_: s_B >= 0

task_B_can_be_reduced_by_non_negative_weeks: r_B >= 0

Task_B_can_be_started_only_after_task_A_is_completed: r_A - s_A + s_B >= 1

task_C_can_be_reduced_at_max_by_0.5_weeks.: r_C <= 0.5

week_at_which_task_C_starts_should_be_non_negative_: s_C >= 0

task_C_can_be_reduced_by_non_negative_weeks: r_C >= 0

Task_C_can_be_started_only_after_task_A_is_completed: r_A - s_A + s_C >= 1

task_D_can_be_reduced_at_max_by_0.0_weeks.: r_D <= 0

week_at_which_task_D_starts_sho

In [15]:
# Solve our problem
prob.solve(solver=pulp.solvers.PULP_CBC_CMD())
#prob.solve(solver=pulp.solvers.COIN_CMD())
#prob.solve(solver=pulp.solvers.GLPK_CMD())
prob.writeLP(filename = "./optimize-project-cost.lp")
solverStatus = pulp.LpStatus[prob.status]
print("Status of solver ", solverStatus)
print("Is problem a Mixed Integer Linear programming ?: ", ['No','Yes'][prob.isMIP()])
#pint("Solver used: ", prob.solver.path)

('Status of solver ', 'Optimal')
('Is problem a Mixed Integer Linear programming ?: ', 'Yes')


In [16]:
if solverStatus == "Optimal":
    # Print objective function value
    print("Minimum increase in project cost with project deadline as K: " + str(K) + " is " \
          + str(pulp.value(prob.objective)))
else:
    print("Problem could not be solved. SolverStatus:" + solverStatus )

Minimum increase in project cost with project deadline as K: 9.5 is 570.0


In [17]:
if solverStatus == "Optimal":
    # Print values of decision variables
    df['task_start_at_week'] = [s[task].varValue for task in df.index]
    df['task_duration_reduced_by'] = [r[task].varValue for task in df.index]
    display(df)
else:
    print("Problem could not be solved. SolverStatus:" + solverStatus )

,prerequisite_tasks,normal_duration_weeks,normal_cost_dollars,crashing_duration_weeks,crashing_cost_dollars,crash_cost_per_week,max_reduction_duration_weeks,task_start_at_week,task_duration_reduced_by
task,,,,,,,,,
A,,1.0,100,0.5,140,80.0,0.5,0.0,0.0
B,A,3.0,150,1.0,270,60.0,2.0,1.0,2.0
C,A,1.0,120,0.5,160,80.0,0.5,1.0,0.0
D,"B,C",1.0,10,1.0,10,0.0,0.0,2.0,0.0
E,D,1.0,225,0.5,300,150.0,0.5,8.5,0.0
F,D,3.0,500,2.0,700,200.0,1.0,3.0,1.0
G,D,3.0,400,1.0,500,50.0,2.0,3.0,2.0
H,G,1.0,150,0.5,170,40.0,0.5,4.0,0.0
I,"F,H",1.5,75,0.5,135,60.0,1.0,5.0,1.0


If we change type of decision variable r from Continuous to Integer, we see that its a Mixed Integer Programming Problem.


## Resources

1. A Blending Problem - https://pythonhosted.org/PuLP/CaseStudies/a_blending_problem.html
2. A Sudoku Problem formulated as an LP - https://pythonhosted.org/PuLP/CaseStudies/a_sudoku_problem.html
3. Linear Programming with Python and PuLP - Part 3, 5 and 6 - http://benalexkeen.com/linear-programming-with-python-and-pulp-part-6/ 
4. Project task details from https://www.linearprogramming.info/formulation-and-resolution-of-a-model-of-linear-programming-to-reduce-the-duration-of-a-project-crashing/
